In [1]:
import os, re, sys, shutil
import subprocess as sp
import random, string
import numpy as np
from arnie.utils import *

In [2]:
from arnie.mfe import mfe
from arnie.free_energy import free_energy

seq = 'CGCUGUCUGUACUUGUAUCAGUACACUGACGAGUCCCUAAAGGACGAAACAGCG'

In [3]:
print('Vienna RNAfold:')
print(mfe(seq))
print(free_energy(seq))

Vienna RNAfold:
((((((.((((((......)))))).......((((.....))))...))))))
-15.92


In [4]:
print('\nLinearFold-V:')
print(mfe(seq, linear=True))
print(free_energy(seq, linear=True))


LinearFold-V:
((((((.((((((......)))))).......((((.....))))...))))))


ValueError: could not convert string to float: 'Coefficient:'

In [5]:
import os, re, sys
import subprocess as sp
import random, string
import numpy as np
from arnie.utils import *
from arnie.pfunc import pfunc

In [8]:
package_locs = load_package_locations()

In [9]:
seq = 'CGCUGUCUGUACUUGUAUCAGUACACUGACGAGUCCCUAAAGGACGAAACAGCG'
pkg = 'vienna'
bpps = False
return_free_energy = False 
beam_size = 100
DEBUG = False

In [27]:
LOC = package_locs['linearpartition']
package='contrafold'
threshknot = False
tmp_file = filename()
tmp_command = filename()
pf_only = int(bpps)

if threshknot:
    TK = '-T'
else:
    TK = '_'
    


In [28]:
command=['echo %s | %s/linearpartition_%s' % (seq, LOC, package[0].lower()), str(beam_size),
 '0', '0', tmp_file, "''", str(pf_only), '0.000001', "''", "''", "''",'%s' % (TK), "''", "''", "''", "''", "''"] # change the command to linearpartition instead of linearpartition_v or linearpartition_c

In [39]:

def pfunc_linearpartition_(seq, bpps=False, package='contrafold', beam_size=100, return_free_energy=False, DEBUG=False, threshknot=False):
    LOC = package_locs['linearpartition']
    tmp_file = filename()
    tmp_command = filename()

    if bpps:
        pf_only = 0
    else:
        pf_only = 1

    if threshknot:
        TK = '-T'
    else:
        TK = '_'

        
    if package == 'vienna':
        vienna_flag = '-V'
    else:
        vienna_flag = ''    
    # args: beamsize, is_sharpturn, is_verbose, bpp_file, bpp_prefix, pf_only, bpp_cutoff,
    #forest_file, mea, gamma, TK, threshold, ThreshKnot_prefix, MEA_prefix, MEA_bpseq

    # args: beamsize, is_sharpturn, is_verbose, bpp_file, bpp_prefix, pf_only, bpp_cutoff,
    #forest_file, mea, gamma, TK, threshold, ThreshKnot_prefix, MEA_prefix, MEA_bpseq, shape_file_path

    #threshknot threshold set to default 0.3
    ##FIXED THE BUG
    # command=['echo %s | %s/linearpartition_%s' % (seq, LOC, package[0].lower()), str(beam_size),
    #  '0', '0', tmp_file, "''", str(pf_only), '0.000001', "''", "''", "''",'%s' % (TK), "''", "''", "''", "''", "''"]
    command = ['echo %s | %s/linearpartition %s' % (seq, LOC, vienna_flag), str(beam_size),
               '0', '0', tmp_file, "''", str(pf_only), '0.000001', "''", "''", "''", '%s' % (TK), "''", "''", "''", "''", "''"]


    with open('%s.sh' % tmp_command,'w') as f:
        f.write(' '.join(command))

    if DEBUG: print(' '.join(command))

    meta_command = ['chmod +x %s.sh; %s.sh' % (tmp_command, tmp_command)]
    p = sp.Popen(meta_command, stdout=sp.PIPE, stderr=sp.PIPE,shell=True)

    stdout, stderr = p.communicate(input=str.encode(seq))

    if DEBUG:
        print('stdout')
        print(stdout)
        print('stderr')
        print(stderr)

    if p.returncode:
        raise Exception('LinearPartition failed: on %s\n%s' % (seq, stderr))

    os.remove("%s.sh" % tmp_command)
    # Note: the linearfold exec says this is free energy in kcal/mol.
    # this is still just cfold log Z

    # linearfold returns two different things depending on which package

    if bpps:
        return 0, tmp_file
    else:

        if package in ['contrafold','eternafold']:
            logZ=float(stderr.decode('utf-8').split(' ')[-1])
            #os.remove('_') # tmp file written by linearpartition forest
            if return_free_energy:
                return -1*logZ, None
            else:
                return np.exp(logZ), None

        elif package=='vienna':
            free_energy = float(stderr.decode('utf-8').split(' ')[-2])
            T=37
            #os.remove('_') # tmp file written by linearpartition forest
            if return_free_energy:
                return free_energy, None
            else:
                return np.exp(-1*free_energy/(.0019899*(273+T))), None

In [40]:
Z, tmp_file = pfunc_linearpartition_(seq, package='contrafold', bpps=bpps, beam_size=beam_size,
    return_free_energy=return_free_energy, DEBUG=DEBUG)

In [41]:
Z

8.743318546631311e+02

In [42]:
tmp_file